# Datamining Program for Procurement

The following piece of code acquires a data frame for all procurement contracts related to a single keyword. The first entry makes a function to get all the links for the contracts.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

driver =  webdriver.Chrome('/Users/vatsalbajaj/Downloads/chromedriver')
old_links = []
def get_links(num):
    try:
        website_name = 'https://eprocure.gov.in/cppp/resultoftenders/byJWFieSVic29mdHdhcmVieSVjYnklZHNlbGVjdGJ5JWUyMDE3YnklZlB1Ymxpc2hlZA==/page='
        page_num = str(num)
        driver.get(website_name + page_num)
        table = driver.find_element_by_id('edit-l-result-teners')
        tender = table.get_attribute('innerHTML')
        semi_links = [m.start() for m in re.finditer('a href="/cppp/aocfullview', tender)]
        for semilink in semi_links:
            old_links.append('https://eprocure.gov.in' + tender[semilink + 8: semilink+38])
    except:
        captcha_URL = driver.current_url
        driver.get(captcha_URL)
        captcha_problem = driver.find_element_by_xpath("//form/div/fieldset/div/div[2]/span").text
        answer = int(captcha_problem[0:2]) + int(captcha_problem[-4:-2])
        inputting_location = driver.find_element_by_xpath("//form/div/fieldset/div/div[2]/input")
        inputting_location.send_keys(answer)
        driver.find_element_by_xpath("//form/div/input").click()
        get_links(num)

The next entry makes a list of links and prints them out.

In [ ]:
for x in range(1,36):
    get_links(x)
links = []
for elem in old_links:
    link = elem.replace('"', '')
    links.append(link)

The next entry creates a utility function to manipulate date and time, and it also creates a contract class.

In [ ]:
import datetime

def convert_date(date):
        dt_obj = (datetime.datetime.strptime(date, "%d-%b-%Y %I:%M %p"))
        unix_time = dt_obj.timestamp()
        return unix_time
    
class Contract():
    
    def __init__(self, organisation_name, tender_description, tender_type, number_of_bids, name_of_selected_bidder, contract_value, published_date, contract_date, completion_date):
        self.organisation_name = organisation_name.text
        self.tender = tender_description.text
        self.type = tender_type.text
        self.number = number_of_bids.text
        self.winner = name_of_selected_bidder.text
        self.value =  contract_value.text
        self.published_date = published_date.text
        self.contract_date = contract_date.text
        self.completion_date = completion_date.text

The next function mines data from every single contract. It saves all the contract objects in a list called information.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

driver = webdriver.Chrome('/Users/vatsalbajaj/Downloads/chromedriver')
information = []
def getInfo(link):
    try:
        str1 = link
        driver.get(str1)
        organisation_name = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[1]/td[@class = 'td_view_field']")
        tender_description = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[3]/td[@class = 'td_view_field']")
        tender_type = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[5]/td[@class = 'td_view_field']")
        number_of_bids = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[6]/td[@class = 'td_view_field']")
        name_of_selected_bidder = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[7]/td[@class = 'td_view_field']")
        contract_value = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[9]/td[@class = 'td_view_field']")
        published_date = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[10]/td[2]")
        contract_date = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[10]/td[4]")
        completion_date = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[11]/td[2]")
        information.append(Contract(organisation_name, tender_description, tender_type, number_of_bids, name_of_selected_bidder, contract_value, published_date, contract_date, completion_date))
    except:
        captcha_URL = driver.current_url
        driver.get(captcha_URL)
        captcha_problem = driver.find_element_by_xpath("//form/div/fieldset/div/div[2]/span").text
        answer = int(captcha_problem[0:2]) + int(captcha_problem[-4:-2])
        inputting_location = driver.find_element_by_xpath("//form/div/fieldset/div/div[2]/input")
        inputting_location.send_keys(answer)
        driver.find_element_by_xpath("//form/div/input").click()
        getInfo(link)
for link in links:
    getInfo(link)

## Load Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

Data cleaning process for every variable. First, we clean the tender type column with only the relevant tender types remaining.

In [ ]:
tendertype_unique = []
for contract in information:
    tendertype_unique.append(contract.type)
print(list(set(tendertype_unique)))
tendertype = []
for tender in tendertype_unique:
    if tender == '' or tender == 'STE(OEM/OES)-SRM' or tender == '2' or tender == 'Single Tender(Urg-M)' or tender == '1' or tender == 'NA':
        tendertype.append(None)
    elif tender == 'Limted' or tender == 'LIMITED TENDER-SRM' or tender == 'LIMITED':
        tendertype.append('Limited')
    else:
        tendertype.append(tender)
print(list(set(tendertype)))
    

This function creates a dataframe and exports it.

In [ ]:
from pandas import DataFrame
contracts = []
i = 0
for contract in information:
    if isinstance(contract.type, str) != True:
        contract.type = null
for contract in information:
    contracts.append([contract.organisation_name, contract.tender, tendertype[i]
                      , contract.number, contract.winner, contract.value, contract.published_date, 
                      contract.contract_date,  contract.completion_date])
    i += 1

    


In [ ]:
df = pd.DataFrame(contracts, columns = ["Organisation name", "Tender description", "Tender type", "Number of bids", 
                                     "Name of selected bidder", "Contract value", "Published date", "Contract date"
                                     , "Completion date"])
df['Contract date'] = pd.to_datetime(df['Contract date'])  
df['Published date'] = pd.to_datetime(df['Published date'])  
df['Completion date'] = pd.to_datetime(df['Completion date'])
df['Number of bids'] = pd.to_numeric(df['Number of bids'], downcast = 'unsigned')
df['Contract value'] = pd.to_numeric(df['Contract value'])
df["Tender type"] = df["Tender type"].astype('category')

for completiondate in df['Completion date']:
    if isinstance(completiondate, dt.datetime) != True:
        completiondate = None


In [ ]:
df.to_csv(r'/Users/vatsalbajaj/desktop/ccs/Procurement Research/procurement_data.csv')